# Secure File Transfer with Authentication and Hybrid Encryption
-- RSA for Key exchange
-- AES for file encryption
-- Hashed password authentication using bcrypt

In [ ]:
#install required libraries
%pip install cryptography bcrypt

In [1]:
#Step1: Import Libraries and Utility Functions

import bcrypt
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding

#RSA Key Generation
def generate_rsa_key():
    private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
    public_key = private_key.public_key()
    return private_key, public_key

#RSA encryption/decryption
def rsa_encrypt(data, public_key):
    return public_key.encrypt(
        data,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
def rsa_decrypt(data, private_key):
    return private_key.decrypt(
        data,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

#AES key and encryption/decryption
def generate_aes_key():
    return Fernet.generate_key()

def aes_encrypt(data, key):
    return Fernet(key).encrypt(data)

def aes_decrypt(data, key):
    return Fernet(key).decrypt(data)

#Password hashing and verification
def hash_password(password):
    return bcrypt.hashpw(password.encode(), bcrypt.gensalt())

def verify_password(password, hashed):
    return bcrypt.checkpw(password.encode(), hashed)

In [2]:
#Step2: Generate Keys and Credentials
private_key, public_key = generate_rsa_key()

#Generate AES key
aes_key = generate_aes_key()

#User credentials
username = "ADMIN"
password = "SecurePassword123"
hashed_password = hash_password(password)

#Simulate user database (in memory)
user_db = {username: hashed_password}

In [3]:
#Step3: Simulate Client Side
#Encrypt AES key with RSA public key
#Encrypt file with AES key

file_content = b'This is TOP-SECRET'

encrypted_aes_key = rsa_encrypt(aes_key, public_key)

encrypted_file = aes_encrypt(file_content, aes_key)

In [4]:
#Simulate Server Side
#Authenticate user
#Decrypt AES key with RSA private key
#Decrypt file content

received_username = "ADMIN"
received_password = "SecurePassword123"

#authenticate 
if received_username in user_db and verify_password(received_password, user_db[received_username]):
    print("User authenticated.")

    #decrypt AES key
    decrypted_aes_key = rsa_decrypt(encrypted_aes_key, private_key)

    #decrypt file content
    decrypted_file = aes_decrypt(encrypted_file, decrypted_aes_key)
    print("Decrypted file content:", decrypted_file.decode())
else:
    print("Authentication failed.")

User authenticated.
Decrypted file content: This is TOP-SECRET
